## Import all the relevant Modules 

In [1]:
%matplotlib inline


#TODO: Rename module shortcuts such as hb into something usefull after development
import Modules.M05_Dynamical_Correlations.Module_Dynamical_Hubbard as dh
from Modules.M00_General.Module_Widgets_and_Sliders import Save_Figure_Button, Click_Save_Figure, set_filename

# default Jupyter widgets
import ipywidgets as widgets
from ipywidgets import HBox, VBox

# Plotting
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':14})

# for printlenght and save figures button
import functools
import numpy as np
np.set_printoptions(linewidth=150) #set output length, default=75

def close_widgets(DIR) -> None:
    """Close all widgets `wi` and displays `di` in notebook directory `dir()`.
    Also clear `Save_Figure_Button`
    """
    for i in range(100):
        if f"w{i}" in DIR:
            exec(f"w{i}.close()")
        if f"d{i}" in DIR:
            exec(f"d{i}.close()")
    # clear `Save_Figure_Button` callbacks, otherwise all previous callbacks are executed and figures are saved multiple times
    Save_Figure_Button._click_handlers.callbacks = []
            

%load_ext autoreload
%autoreload 2

# Dynamical Correlation Functions
<!---  Define a few convenience macros for bra-ket notation. -->
$\newcommand{\ket}[1]{\left\vert{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right\vert}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\dyad}[2]{\left|{#1}\middle\rangle\middle\langle{#2}\right|}$
$\newcommand{\mel}[3]{\left\langle{#1}\vphantom{#2#3}\right|{#2}\left|{#3}\vphantom{#1#2}\right\rangle}$
$\newcommand{\expval}[1]{\left\langle{#1}\right\rangle}$
$\newcommand\dif{\mathop{}\!\mathrm{d}}$
$\newcommand\ii{\mathrm{i}}$
$\newcommand{\coloneqq}{\mathop{:=}}$
$\newcommand{\abs}[1]{\left\vert{#1}\right\vert}$
$\newcommand{\vb}[1]{\mathbf{#1}}$
$\newcommand{\im}[1]{\operatorname{Im}{#1}}$
$\newcommand{\re}[1]{\operatorname{Re}{#1}}$

Up until now we only looked at equal-time correlation functions. We want to further look at so-called **dynamical correlation functions** which depend on time $t$ or frequency $\omega$. To this end we define the time dependent correlation function
$$ \large
    \expval{\hat{A}(t)\hat{B}(0)} \coloneqq \mel{\psi_g}{e^{\ii t \hat{H}} \hat{A} e^{-\ii t \hat{H}}  \hat{B}}{\psi_g},
$$
where $\hat{A}$ and $\hat{B}$ are arbitrary operators and $\hat{A}(t) \coloneqq e^{\ii t \hat{H}} \hat{A} e^{-\ii t \hat{H}}$ is the time evolution of the operator $\hat{A}$.


## Spectral Representation of the Correlation Function
By using the completeness relation of the Hamiltonian's eigenfunctions $\ket{n}$, i.e. $1 = \sum_n \dyad{n}{n}$ and the eigenvalue equation for $H$, we can rewrite the correlation function as

$$ \large
\begin{align*} 
	\expval{\hat{A}(t)\hat{B}(0)} &\coloneqq \mel{\psi_g}{e^{\ii t \hat{H}} \hat{A} e^{-\ii t \hat{H}}  \hat{B}}{\psi_g} \\
	%
	&= \sum_n  \mel{\psi_g}{e^{\ii t \hat{H}} \hat{A} \dyad{n}{n} e^{-\ii t \hat{H}}  \hat{B}}{\psi_g} \\
	%
	&= \sum_n e^{-\ii t (E_n - E_g)} \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g} \\
	%
	&= \sum_n e^{-\ii t \bar{E}_n} \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g},
\end{align*}
$$

where we defined the energy difference $\bar{E}_n \coloneqq E_n - E_g$.
Note that we drastically simplified the expression from the complicated exponential of an operator to an exponential of just a number.
As we have the sum of exponentials, it is natural to look at the Fourier transform of the correlation function.

## Fourier Transform of the Correlation Function
If one were to naively apply the Fourier transform from time domain $t$ to frequency domain $\omega$ of the correlation function, one would obtain an expression
$$ \large
    \propto \int_{-\infty}^{\infty} \dif{t} \, e^{-\ii t (\omega - \bar{E}_n)}, 
$$
which is unfortunately not well defined as a function, but only in a distributional sence. We will remedy this problem in measurability in a twofold way.
First, we will restrict the time range to the interval $[0, \infty)$.
From a physical point of few this implies that we are only interested in the correlation of the system were Operator $\hat{A}$ is applied after operator $\hat{B}$ was applied. 
One calls this the **causal Green's function** 

$$ \large
    G_{AB}(t) \coloneqq \Theta(t) \mel{\psi_g}{e^{\ii t \hat{H}} \hat{A} e^{-\ii t \hat{H}}  \hat{B}}{\psi_g} = \Theta(t) \sum_n e^{-\ii t \bar{E}_n} \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g}.
$$,
where $\Theta(t)$ is the Heaviside step function.

The second step is to promote $\omega$ to a complex number, i.e. $\omega \to \Omega \coloneqq \omega + \ii \delta$. We can therefore evaluate the Fourier transformation of the causal Green's function as

$$ \large
    \int_{-\infty}^{\infty} \dif{t} \, \Theta(t) \, e^{-\ii t (\omega - \bar{E}_n)} =  \int_{0}^{\infty} \dif{t} \, e^{-\ii t (\omega - \bar{E}_n)} = \frac{1}{\omega + \ii \delta - \bar{E}_n} \quad \mathrm{for} \quad \delta > 0.
$$

To finally obtain the Fourier transform of the causal Green's function we have to take the limit $\delta \to 0$ from above, which gives (keeping in mind, that a distribution is only defined via integration by a test function)

$$ \large
    \lim_{\delta \to 0} \frac{1}{\omega + \ii \delta - \bar{E}_n} = \mathrm{P.V.} \left(\int_{-\infty}^{\infty} \dif{\omega} \,\frac{1}{\omega - \bar{E}_n}\right) - \ii \pi \delta(\omega - \bar{E}_n),
$$

where $\mathrm{P.V.}$ denotes the principal value of an integral. 
This results in our final result for the Fourier transform of the causal Green's function

$$ \large
	\tilde{G}_{AB}(\omega) \coloneqq \expval{\hat{A}\hat{B}}_\omega \coloneqq \sum_n \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g} \left( \mathrm{P.V.} \left(\int_{-\infty}^{\infty} \dif{\omega} \,\frac{1}{\omega - \bar{E}_n}\right) - \ii \pi \delta(\omega - \bar{E}_n)\right).
$$

## Numerical Approximation of $\tilde{G}_{AB}(\omega)$

Since neither the principal value nor the delta distribution are well defined in a numerical sense, to visualize them we have to approximate them by using **Lorentzian curves** for the real and imaginary part of $\tilde{G}_{AB}(\omega)$. 
These Lorentzian curves are thus defined as
$$
\large
\begin{align*} 
	\im{\tilde{G}_{AB}(\omega)} &\approx \sum_n  \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g} \left( \frac{\delta / 2}{(\omega - \bar{E}_n)^2 + \delta^2} \right) \\
	%
	\re{\tilde{G}_{AB}(\omega)} &\approx \sum_n  \mel{\psi_g}{ \hat{A} }{n}\mel{n} {\hat{B}}{\psi_g} \left( \frac{\bar{E}_n - \omega }{(\omega - \bar{E}_n)^2 + \delta^2} \right),
\end{align*}
$$
where $\delta \ll 1$ is a small parameter that assures finite width of the Lorentzian curves.

In [2]:
# create instance of DynamicalHubbard class
h1 = dh.DynamicalHubbard()

# layout of widgets
box_layout = widgets.Layout(border='solid 2px')

## Spin-Spin Correlation Functions

In the two widgets below you can explore the real an imaginary part of the Fourier transformed causal Green's function of the spin-spin correlation for the Hubbard model. Due to symmetry reasons, some combinations of $i$ and $j$ are equivalent, e.g. for $n=6$ we have $\expval{\hat{S_1}\hat{
S_2}}_\omega = \expval{\hat{S_1}\hat{S_6}}_\omega$, which is why we only show the minimal subset containing all different combinations

By changing the value of $U$ the spectral weight is shifted to higher energies, while the width of the Lorentzian curves is determined by the value of $\delta$. Setting $\delta = 0$ would result in a delta distribution, which is not well-defined in a numerical sense, and will instead set $\delta = 10^{-5}$.

One can adjust the range of $\omega$ via the range slider, which allows plotting over negative values of $\omega$. Due to the symmetry of the Hubbard model, the spectral weight is symmetric around $\omega = 0$, which can be explored by the user.

### Real-Space

In [3]:
close_widgets(dir())

#create the widget
w1 = widgets.interactive(lambda **kwargs: h1.Plot_G_SzSz(Lorentzian="Imaginary"), w=h1.omega_range, u=h1.u25, d=h1.delta, n=h1.n, s_up=h1.s_up, s_down=h1.s_down, box=h1.t_ij);

# create the save figure button
filename = set_filename("Dynamical_Spin-Spin_Correlation_Imaginary.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w1, name_widget=filename, output=h1.out, path="Figures/"))

# vertical and horizontal box for widgets
d1 = VBox([
    	HBox([Save_Figure_Button, filename, h1.out], layout=box_layout),
        HBox([
            VBox(w1.children[0:3], layout=box_layout),
			VBox(w1.children[3:6], layout=box_layout),
			VBox([w1.children[6]], layout=box_layout)
			]),
        w1.children[-1],
		])

display(d1)
w1.update()	# otherwise graph will only appear after the first interaction

In [4]:
close_widgets(dir())

#create the widhet
w2 = widgets.interactive(lambda **kwargs: h1.Plot_G_SzSz(Lorentzian="Real"), w=h1.omega_range, u=h1.u25, d=h1.delta, n=h1.n, s_up=h1.s_up, s_down=h1.s_down, box=h1.t_ij);

# create the save figure button
filename = set_filename("Dynamical_Spin-Spin_Correlation_Real.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w2, name_widget=filename, output=h1.out, path="Figures/"))

# vertical and horizontal box for widgets
d2 = VBox([
    	HBox([Save_Figure_Button, filename, h1.out], layout=box_layout),
        HBox([
            VBox(w2.children[0:3], layout=box_layout),
			VBox(w2.children[3:6], layout=box_layout),
			VBox([w2.children[6]], layout=box_layout)
			]),
        w2.children[-1],
		])

display(d2)
w2.update()	# otherwise graph will only appear after the first interaction

### Reciprocal-Space

Like in the previous notebook we can use translation symmetry to reduce the number of different real-space correlation functions ($n^2$) we have to calculate to just $\left \lfloor n/2 \right \rfloor$ non-zero, reciprocal-space functions, holding all the information. Of course one can switch back and forth from one representation to the other via a Fourier transform.

The widget below shows the imaginary and real part of all relevant time-dependent, reciprocal-space correlation functions $\expval{S_z(\vb k) S_z(-\vb k)}_\omega$ as a function of the interaction strength $U$. Note that in this case we have no symmetry around $\omega = 0$. Also, for $k=0 \cdot \pi$ the correlation function is identically zero.

In [6]:
close_widgets(dir())

#create the widhet
w3 = widgets.interactive(h1.Plot_G_SzkSzk, w=h1.omega_range, u=h1.u25, d=h1.delta, n=h1.n, s_up=h1.s_up, s_down=h1.s_down, box=h1.t_ij);

# create the save figure button
filename = set_filename("Reciprocal_Dynamical_Spin-Spin_Correlation.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w3, name_widget=filename, output=h1.out, path="Figures/"))

# vertical and horizontal box for widgets
d3 = VBox([
    	HBox([Save_Figure_Button, filename, h1.out], layout=box_layout),
        HBox([
            VBox(w3.children[0:3], layout=box_layout),
			VBox(w3.children[3:6], layout=box_layout),
			VBox([w3.children[6]], layout=box_layout)
			]),
        w3.children[-1],
		])

display(d3)
w3.update()	# otherwise graph will only appear after the first interaction

## Electronic Propagators

At the beginning of these notebooks we introduced a lot of symmetry concepts and conservation laws to reduce the computational effort. 
For example, in the Hubbard Hamiltonian we have the conservation of the total number of particles $N$, the total spin $S^2$, as well as conservation of the spin component $S_z$.
These allowed us to basically reduce the dimension of the Hilbert space to significantly less than $2^N$. In terms of matrices this means that we can diagonalise the Hamiltonian in a block-wise fashion, where each block corresponds to a certain number of spin-up and spin-down particles. There are $N^2$ blocks in total due to the different combinations of spin up and spin down particles.

Up until now it was sufficient to only look at one of these sectors, but now we want to look at electronic propagators of the form

$$
\large
\expval{\hat{c}_{j,\uparrow}(t)\hat{c}^\dagger_{i,\uparrow}(0)} \coloneqq \mel{\psi_g}{e^{\ii t \hat{H}} \hat{c}_{j,\uparrow} e^{-\ii t \hat{H}}  \hat{c}^\dagger_{i,\uparrow}}{\psi_g},
$$
also known as one-particle correlation function or one-particle propagator, that temporarily change the number of electrons in the system. Here $\hat{c}_{j,\uparrow}$ is the annihilation operator for an electron with spin up at site $j$ and $\hat{c}^\dagger_{i,\uparrow}$ is the creation operator for an electron with spin up at site $i$. Thus, the operator $\hat{c}_{j,\uparrow} \hat{c}^\dagger_{i,\uparrow}$ creates an electron at site $i$ and annihilates it at site $j$ after time $t$.

After applying the creation/annihilation operator, the number of electrons in the system is changed by $\pm 1$, which means that we have to look at a different sector in the Hamiltonian's matrix representation to perform the time evolution $\exp(-\ii t \hat{H})$. Thus we also have to calculate the matrix representation and the basis states for these two other sectors.

### Creation and Annihilation Operators

All the operators we encountered so far were represented by square matrices, which is not (necessarily) the case for the creation and annihilation operators. As mentioned above this is the first time we have to deal with operators that change the number of particles in the system and thus move us from one sector to another. To better illustrate this, let us look at a $n=3$ sited system with $s_\mathrm{up} = 1$ and $s_\mathrm{down} = 2$ particles. There are 9 possibly basis states in this system. The spin up creation and annihilation operators (in our convention) are thus given by

$$
\begin{alignat*}{3}
\hat{c}_{1,\uparrow} &= \begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{pmatrix},& \;
%
\hat{c}_{2,\uparrow} &= \begin{pmatrix}
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0
\end{pmatrix},& \;
%
\hat{c}_{3,\uparrow} &= \begin{pmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0
\end{pmatrix}\;
\\
\\
\hat{c}^\dagger_{1,\uparrow} &= \begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0
\end{pmatrix},& \;
%
\hat{c}^\dagger_{2,\uparrow} &= \begin{pmatrix}
-1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{pmatrix},& \;
%
\hat{c}^\dagger_{3,\uparrow} &= \begin{pmatrix}
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
  0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0

\end{pmatrix}
\end{alignat*}
$$

Notice how this sector's annihilation operators are not square matrices, but instead have only 3 rows and 9 columns. This is because there are only three basis states in the sector with one spin up particle less. The creation operators in this case are 9x9 square matrices, since there are 9 basis states in the sector with one spin up particle more.

We can also observe that some rows and columns are zero, which indicates that the corresponding basis states at position $i$ already contains an electron, and we cannot add another one or position $i$ does not contain an electron, and we thus cannot remove one. 

### Reciprocal-Space

As always, we can also Fourier Transform these operators via

$$
\large
\begin{align*}
 	\hat{c}_{\uparrow}(\vb k) &:= \frac{1}{\sqrt{N}}\sum_{\vb k} \exp(-\ii \vb k \cdot \vb R) \hat{c}_{i,\uparrow},\\
	\hat{c}^\dagger_{\uparrow}(\vb k') &:= \frac{1}{\sqrt{N}}\sum_{\vb k'} \exp(\ii \vb k' \cdot \vb R) \hat{c}^\dagger_{i,\uparrow},
\end{align*}
$$
to obtain the reciprocal-space representation of the creation and annihilation operators, which again reduces the number of non-zero Green's functions we have to deal with from $n^2$ correlations of the form $\expval{\hat{c}_{j,\uparrow}\hat{c}^\dagger_{i,\uparrow}}_\omega $ to only $\left \lfloor n/2 \right \rfloor$ of the form $\expval{\hat{c}(k)\hat{c}^\dagger(k)}_\omega$.
Here we already implicitly assumed that the system is translationally invariant such that this time $k' = k$ dude to the conjugation of the Fourier transform for the daggered creation operator.

For the sake of completeness, we also show the reciprocal-space representation of the spin up creation and annihilation operators for the same system with three sites:


$$
\begin{alignat*}{3}
\hat{c}_{\uparrow}(0) &= \begin{pmatrix}
 a & 0 & 0 & a & 0 & 0 & a & 0 & 0 \\
 0 & a & 0 & 0 & a & 0 & 0 & a & 0 \\
 0 & 0 & a & 0 & 0 & a & 0 & 0 & a \\
\end{pmatrix},& \;
%
\hat{c}_{\uparrow}(1) &= \begin{pmatrix}
b & 0 & 0 & \overline{b} & 0 & 0 & a & 0 & 0 \\
 0 & b & 0 & 0 & \overline{b} & 0 & 0 & a & 0 \\
 0 & 0 & b & 0 & 0 & \overline{b} & 0 & 0 & a \\
\end{pmatrix},& \;
%
\hat{c}_{\uparrow}(2) &= \begin{pmatrix}
\overline{b} & 0 & 0 & b & 0 & 0 & a & 0 & 0 \\
 0 & \overline{b} & 0 & 0 & b & 0 & 0 & a & 0 \\
 0 & 0 & \overline{b} & 0 & 0 & b & 0 & 0 & a \\
\end{pmatrix}\;
\\
\\
\hat{c}^\dagger_{\uparrow}(0) &= \begin{pmatrix}
 -a & 0 & 0 & a & 0 & 0 & 0 & 0 & 0 \\
 0 & -a & 0 & 0 & a & 0 & 0 & 0 & 0 \\
 0 & 0 & -a & 0 & 0 & a & 0 & 0 & 0 \\
 -a & 0 & 0 & 0 & 0 & 0 & a & 0 & 0 \\
 0 & -a & 0 & 0 & 0 & 0 & 0 & a & 0 \\
 0 & 0 & -a & 0 & 0 & 0 & 0 & 0 & a \\
 0 & 0 & 0 & -a & 0 & 0 & a & 0 & 0 \\
 0 & 0 & 0 & 0 & -a & 0 & 0 & a & 0 \\
 0 & 0 & 0 & 0 & 0 & -a & 0 & 0 & a \\
\end{pmatrix},& \;
%
\hat{c}^\dagger_{\uparrow}(1) &= \begin{pmatrix}
 -b & 0 & 0 & \overline{b} & 0 & 0 & 0 & 0 & 0 \\
 0 & -b & 0 & 0 & \overline{b} & 0 & 0 & 0 & 0 \\
 0 & 0 & -b & 0 & 0 & \overline{b} & 0 & 0 & 0 \\
 -a & 0 & 0 & 0 & 0 & 0 & \overline{b} & 0 & 0 \\
 0 & -a & 0 & 0 & 0 & 0 & 0 & \overline{b} & 0 \\
 0 & 0 & -a & 0 & 0 & 0 & 0 & 0 & \overline{b} \\
 0 & 0 & 0 & -a & 0 & 0 & b & 0 & 0 \\
 0 & 0 & 0 & 0 & -a & 0 & 0 & b & 0 \\
 0 & 0 & 0 & 0 & 0 & -a & 0 & 0 & b \\
\end{pmatrix},& \;
%
\hat{c}^\dagger_{\uparrow}(2) &= \begin{pmatrix}
-\overline{b} & 0 & 0 & b & 0 & 0 & 0 & 0 & 0 \\
 0 & -\overline{b} & 0 & 0 & b & 0 & 0 & 0 & 0 \\
 0 & 0 & -\overline{b} & 0 & 0 & b & 0 & 0 & 0 \\
 -a & 0 & 0 & 0 & 0 & 0 & b & 0 & 0 \\
 0 & -a & 0 & 0 & 0 & 0 & 0 & b & 0 \\
 0 & 0 & -a & 0 & 0 & 0 & 0 & 0 & b \\
 0 & 0 & 0 & -a & 0 & 0 & \overline{b} & 0 & 0 \\
 0 & 0 & 0 & 0 & -a & 0 & 0 & \overline{b} & 0 \\
 0 & 0 & 0 & 0 & 0 & -a & 0 & 0 & \overline{b} \\
\end{pmatrix}\;
\end{alignat*}
$$
Here we used the abbreviations $a = \frac{1}{\sqrt{3}} \approx 0.58$, $b = \frac{1}{2\sqrt{3}}(\ii-\sqrt{3})\approx -0.29 - 0.5\ii$ and $\overline{b}$ for the complex conjugate of $b$.

## Chemical Potential $\mu$

To account for the fact, that we have now operators that change the number of particles in the system, we have to introduce a new parameter $\mu$, the so-called **chemical potential** and adjust the Hamiltonian to

$$
\large
\hat{H} = -t \sum_{\langle i,j \rangle, \sigma} \hat{c}^\dagger_{i,\sigma} \hat{c}_{j,\sigma} + U \sum_i \hat{n}_{i,\uparrow} \hat{n}_{i,\downarrow} - \mu \sum_{i,\sigma} \hat{n}_{i,\sigma},
$$
where $\hat{n}_{i,\sigma} \coloneqq \hat{c}^\dagger_{i,\sigma} \hat{c}_{i,\sigma}$ is the number operator for spin $\sigma$ at site $i$.
The chemical is the energy required to add an electron to the system, and it essentially shifts the one-particle energy of all states by $\pm\mu$.
Without this parameter, if we were to increase the value of $U$ we might shift the ground state away from the sector we are interested in.
To avoid this, we can adjust the chemical potential to compensate for an increase in $U$ such that the ground state is always in the sector we are interested in by setting $\mu = \frac{U}{2}$.

As with on-site interactions, the third term with the chemical potential $\mu$ is diagonal in the basis of the number of particles.
Thus, we can simply add/subtract $\mu$ from the energies $E_n \mapsto E_n \pm \mu$ in our numerical approximations for the Fourier transform of the causal Green's function $\tilde{G}_{AB}(\omega)$.

## One-Particle Spectral Function
With all the tools developed above we can finally look at the one-particle spectral function $A(\omega)$ defined via

$$
\large
	A(\omega) \coloneqq \begin{cases}
	\sum\limits_l \abs{\mel{n+1, 0}{ \hat{c}^\dagger_{i,\uparrow} }{n, l}}^2 \delta\left(\omega - (E_{n+1}^l - E_{n}^0 - \mu)\right) & \mathrm{for} \quad \omega > 0, \\
	\sum\limits_l \abs{\mel{n-1, 0}{ \hat{c}^\dagger_{i,\uparrow} }{n, l}}^2 \delta\left(\omega - (E_{n-1}^l - E_{n}^0 + \mu)\right) & \mathrm{for} \quad \omega < 0,
\end{cases}
$$
where $\ket{n, l}$ is the $l$-th eigenstate (ordered ascending from the ground state $l=0$) in the sector with $n$ electrons and $E_{n}^l$ is the corresponding energy. Hence, with this convention the electron removal frequency $\omega$ is negative and the electron addition frequency is positive.
We can, as always, also use the reciprocal-space representation of the creation $\hat{c}^\dagger_{\uparrow}(k)$ and annihilation $\hat{c}_{\uparrow}(k)$ operators to obtain the reciprocal-space representation of the spectral function $\tilde{A}(\omega, k)$

In [3]:
from Modules.M05_Dynamical_Correlations.Module_Dynamical_Hubbard import Sector
hp = dh.DynamicalHubbardPropagator()
# layout of widgets
box_layout = widgets.Layout(border='solid 2px')
close_widgets(dir())

In the widget below you can observe the Fourier Transform of the one-particle spectral function for the Hubbard model.
One can change the number of sites $n$ and the number of spin-up and spin-down particles in the system, as well as the interaction strength $U$ and the chemical potential $\mu$. The chemical potential is initially set to $\mu = \frac{U}{2}$, which is the value that keeps the ground state in the sector we are interested. Note that by changing mu, we only shift the spectral function to the left or right, but do not change its shape.

The range of $\omega$ can be adjusted via the range slider, which allows plotting over negative values of $\omega$.
We only show the $\left \lfloor n/2 \right \rfloor + 1$ non-zero spectral functions, where $k = k'$, where we plot the contributions from the electron creation operator $\hat{c}^\dagger_{\uparrow}(k)$ in orange and the electron annihilation operator $\hat{c}_{\uparrow}(k)$ in blue separately.

Finally, the last image shows the local one-particle spectral functions for the sites $i$ and $j$, which can also be adjusted via their respective combo boxes.
Note the part attributed to the creation operator is shown in red and the part of the annihilation operator is shown in green.

In [5]:
close_widgets(dir())

#create the widhet
w4 = widgets.interactive(hp.Plot_A_w, w=hp.omega_range, u=hp.u25, d=hp.delta, m=hp.mu, n=hp.n, s_up=hp.s_up, s_down=hp.s_down, box=hp.t_ij, i=hp.site_i, j=hp.site_j);

# create the save figure button
filename = set_filename("One-Particle_Spectral_Function.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w4, name_widget=filename, output=hp.out, path="Figures/"))

# vertical and horizontal box for widgets
d4 = VBox([
    	HBox([Save_Figure_Button, filename, hp.out], layout=box_layout),
        HBox([
            VBox(w4.children[0:4], layout=box_layout),
			VBox(w4.children[4:7], layout=box_layout),
			VBox(w4.children[7:10], layout=box_layout)
			]),
        w4.children[-1],
		])

display(d4)
w4.update()	# otherwise graph will only appear after the first interaction